# Populations creations

In this notebook, we create various populations to be used in later experiments.

In [ ]:
import sys

import json
import sys
import pandas as pd
import matplotlib.pyplot as plt

import tinytroupe
from tinytroupe.agent import TinyPerson
from tinytroupe.environment import TinyWorld
from tinytroupe.factory import TinyPersonFactory
from tinytroupe.validation import TinyPersonValidator
from tinytroupe import control

from tinytroupe.extraction import ResultsExtractor
from tinytroupe.profiling import Profiler

In [ ]:
from tinytroupe.profiling import Profiler

## Common parameters and functions

In [ ]:
small_population_size = 5
population_size = 20
total_population_size = 30

large_population_size = 50
large_total_population_size = 60

In [ ]:
def save_population(population:list, folder_path:str):
    """
    Save the population to a JSON file.
    """
    for person in population:
         person.save_specification(f"{folder_path}/{person.name}.agent.json")
    

In [ ]:
def describe_population_sampling_space(nationality:str, characteristics_file_path:str=None, verbose:bool=True) -> str:
    """
    Describe the sampling space of the population.
    """

    sampling_space_description =\
        f"""
        A uniform random representative sample of people from the {nationality} population. 
        Make sure you consider very detailed, fine-grained, characteristics of the individuals in the population.
        Please consider as many different population segments as possible, while always keeping proportions correct.
        For example, instead of sampling 10 people from segment A and 5 from segment B, you can instead
        sample 2 from A, 1 from B, and 7 others from other segments, provided the proportions are maintained correct.
        """

    if characteristics_file_path:
        with open(characteristics_file_path, 'r') as file:
            characteristics = json.load(file)

        sampling_space_description +=\
            f"""
            Besides anything you know about this population, also consider the following information 
            to sample proportionally to the presence of such groups and characteristics in the population.        
            
            {json.dumps(json.loads(open(characteristics_file_path).read()), indent=4)}
            """
    
    if verbose:
        print(sampling_space_description)
    
    return sampling_space_description

         

    

We'll also use a post-processing function to further adjust the audience to our needs.

In [ ]:
def post_process_agent(agent, group=None):
    pass

    # TODO do we need any of this?
    #
    ## makes agents authentic and honest. Preliminary results show that this induces agents to be a bit more realistic.
    #agent.import_fragment("./fragments/genuine.agent.fragment.json")
    #
    #if group == "families":
    #    agent.import_fragment("./fragments/loving_parent.agent.fragment.json")
    #elif group == "couples":
    #    pass # nothing for now 
    #elif group == "singles":
    #    pass # nothing for now
    #

In [ ]:
profiler = Profiler()

## Population: General Western population sample

In [ ]:
global_general_sampling_space_description = describe_population_sampling_space("Global")

In [ ]:
global_general_population_factory = TinyPersonFactory(sampling_space_description=global_general_sampling_space_description, 
                                                      total_population_size=total_population_size)


In [ ]:
global_general_population_factory.initialize_sampling_plan()
global_general_population_factory.sampling_plan

In [ ]:
global_general_population = global_general_population_factory.generate_people(population_size, 
                                                                             post_processing_func=post_process_agent,
                                                                             verbose=True)

In [ ]:
profiler.profile(global_general_population)

In [ ]:
save_population(global_general_population, "./population/global_general_2/")

## Population: US general population sample

In [ ]:
usa_general_sampling_space_description = describe_population_sampling_space("American", "./information/populations/usa.json")

Summon the population out of thin simulated air. To do so, we build a factory whose context defines the sampling space. Actual agents will be later sampled from this factory.

In [ ]:
us_general_population_factory = TinyPersonFactory(sampling_space_description=usa_general_sampling_space_description, total_population_size=total_population_size)


In [ ]:
us_general_population_factory.initialize_sampling_plan()
us_general_population_factory.sampling_plan

We can request a specific number of people.

In [ ]:
us_general_population = us_general_population_factory.generate_people(population_size, 
                                                                      post_processing_func=post_process_agent,
                                                                      verbose=True)

In [ ]:
profiler.profile(us_general_population)

In [ ]:
save_population(us_general_population, "./population/usa_general_2/")

In [ ]:
TinyPerson.all_agents_names()

In [ ]:
len(TinyPerson.all_agents_names())

In [ ]:
print(list(TinyPerson.all_agents.values())[0].generate_agent_system_prompt())

## Population: US picky and negative population sample

In [ ]:
usa_picky_sampling_space_description = describe_population_sampling_space("American", "./information/populations/usa.json")

Summon the population out of thin simulated air. To do so, we build a factory whose context defines the sampling space. Actual agents will be later sampled from this factory.

In [ ]:
us_picky_population_factory = TinyPersonFactory(sampling_space_description=usa_picky_sampling_space_description, total_population_size=total_population_size)


In [ ]:
us_picky_population_factory.initialize_sampling_plan()
us_picky_population_factory.sampling_plan

We can request a specific number of people.

In [ ]:
us_picky_population = us_picky_population_factory.generate_people(population_size,
                                                                  agent_particularities="A very negative person, doesn't like anything. Very critical and picky, but never helpful. A rude, unfriendly, person. Rather evil too.", 
                                                                      post_processing_func=post_process_agent,
                                                                      verbose=True)

In [ ]:
profiler.profile(us_picky_population)

In [ ]:
save_population(us_picky_population, "./population/usa_picky_2/")

## Populations: US general population sliced by singles, families (children), and couples(no children)

In [ ]:
usa_general_sampling_space_description = describe_population_sampling_space("American", "./information/populations/usa.json")

In [ ]:
us_singles_factory = TinyPersonFactory(sampling_space_description=usa_general_sampling_space_description, total_population_size=large_total_population_size)
us_families_factory = TinyPersonFactory(sampling_space_description=usa_general_sampling_space_description, total_population_size=large_total_population_size)
us_couples_factory = TinyPersonFactory(sampling_space_description=usa_general_sampling_space_description, total_population_size=large_total_population_size)

In [ ]:
us_singles = us_singles_factory.generate_people(large_population_size, 
                                               agent_particularities="Must be single and have no children.", 
                                               post_processing_func=lambda x: post_process_agent(x, group="singles"),
                                               verbose=True)

In [ ]:
profiler.profile(us_singles)
save_population(us_singles, "./population/usa_singles_2/")

In [ ]:
us_families = us_families_factory.generate_people(large_population_size,
                                            agent_particularities="Must be married and have young children (i.e., less than 6 years old ).", 
                                            post_processing_func=lambda x: post_process_agent(x, group="families"),
                                            verbose=True)

In [ ]:
profiler.profile(us_families)
save_population(us_families, "./population/usa_families_2/")

In [ ]:
us_couples = us_couples_factory.generate_people(large_population_size, 
                                         agent_particularities="Must either be married or have a girlfriend/boyfriend. Must have no children.", 
                                         post_processing_func=lambda x: post_process_agent(x, group="couples"),
                                         verbose=True)

In [ ]:
profiler.profile(us_couples)
save_population(us_couples, "./population/usa_couples_2/")

Let's try to make family sampling more directed, to ensure we capture some nuanced behavior needed later.

In [ ]:
usa_general_sampling_space_description_b =\
f"""
{usa_general_sampling_space_description}

# MANDATORY ADDITIONAL CHILDREN AND FAMILY REQUIREMENTS
When computing sampling dimensions for the person's characteristics, you **MUST** also include the following dimensions:
  - "Vacations without children": to what extent the couple takes vacations without their children (e.g., romantic trips, weekend getaways, etc.) or always take their children with them.
  - "Help with child care and support network": to what extent the person is alone in taking care of his/her children or have some sort of help or support network (from family, friends,  neighbors, baby sitters, school, etc.), 
     either a few hours a day (e.g., daycare) or a few days (e.g., grandparents take care of children).
  
The distribuition for these dimensions MUST: be varied; include both common and extreme cases; and reflect their proportional presence in the American population.

"""

print(usa_general_sampling_space_description_b)

In [ ]:

us_families_b_factory = TinyPersonFactory(sampling_space_description=usa_general_sampling_space_description_b, total_population_size=large_total_population_size, 
                                          context="This sample refers to families with young children, so ensure children and family-related characteristics are well represented whenever possible.")


In [ ]:
us_families_b = us_families_b_factory.generate_people(large_population_size,
                                            agent_particularities="Must be married and have young children (i.e., less than 6 years old ). Must have vacation preferences, including how children fit into those.", 
                                            post_processing_func=lambda x: post_process_agent(x, group="families"),
                                            verbose=True)

In [ ]:
#profiler.profile(us_families_b)
save_population(us_families_b, "./population/usa_families_2b/")

In [ ]:

us_families_c_factory = TinyPersonFactory(sampling_space_description=usa_general_sampling_space_description_b, total_population_size=large_total_population_size, enforce_usage_of_all_dimensions=True,
                                          context="This sample refers to families with young children, so ensure children and family-related characteristics are well represented whenever possible.")


In [ ]:
us_families_c = us_families_c_factory.generate_people(large_population_size,
                                            agent_particularities="Must be married and have young children (i.e., less than 6 years old ). Must have vacation preferences, including how children fit into those.", 
                                            post_processing_func=lambda x: post_process_agent(x, group="families"),
                                            verbose=True)

In [ ]:
#profiler.profile(us_families_c)
save_population(us_families_c, "./population/usa_families_2c/")

## Population: Scandinavia and Norway general populations sample

In [ ]:
#scandinavia_general_sampling_space_description = describe_population_sampling_space("Scandinavian", "./information/populations/scandinavia.json")

In [ ]:
norway_general_sampling_space_description = describe_population_sampling_space("Norwegian", "./information/populations/norway.json")

In [ ]:
norway_factory = TinyPersonFactory(sampling_space_description=norway_general_sampling_space_description, total_population_size=total_population_size)
norway_general = norway_factory.generate_people(population_size, verbose=True) 

In [ ]:
profiler.profile(norway_general)
save_population(norway_general, "./population/norway_general_2/")

## Population: Mexican general population sample

In [ ]:
mexico_general_sampling_space_description = describe_population_sampling_space("Mexican", "./information/populations/mexico.json")

In [ ]:
mexico_factory = TinyPersonFactory(sampling_space_description=mexico_general_sampling_space_description, total_population_size=total_population_size)
mexico_general = mexico_factory.generate_people(population_size, verbose=True) 

In [ ]:
profiler.profile(mexico_general)
save_population(mexico_general, "./population/mexico_general_2/")

## Population: Difficult people

In [ ]:
difficult_people_sampling_space_description = \
f""""
A uniform random representative sample of people from a population of difficult people. Here, difficult people are defined via these dimensions:
  - Negativity: people who are very negative, varying from depressed to very angry.
  - Criticism: people who are very critical and picky, but never helpful.
  - General dislike: people who always find excuses to not like something.
  - Pessimism: people who holds a pessimistic view of the world.
  - Experience deprived: people who are rarely willing to try new experiences, products and services.
  - Jealousy: people who detest when others have fun or success, and vary between trying to look superior or simply jealous.
  - Consumer behavior: people who are excessively demanding customers, and are never satisfied with anything.

This target population also have other more pedestrian characteristics that can vary over the usual demographic dimensions, such as:
  - age, from young to old
  - social economic status, from poor to rich
  - occupation, from unemployed to executive
  - education level, from no education to PhD
  - personality traits, from introverted to extroverted, from open to closed, etc., except when this would conflict with the definition of difficult people above.
  - interests, from sports to arts, from science to politics, etc., except when this would conflict with the definition of difficult people above.
  - beliefs, except when this would conflict with the definition of difficult people above.
  - long-term goals, except when this would conflict with the definition of difficult people above.
  - other standard demographic charactersits, as long as they do not conflict with the definition of difficult people above.
  
Make sure you consider very detailed, fine-grained, characteristics of the individuals in the population. In particular, provide a lot of detail
and dimensions on the various ways the people are difficult, and how this varies across the population.
Please consider as many different population segments as possible.
""" 

In [ ]:
difficult_people_population_factory = TinyPersonFactory(sampling_space_description=difficult_people_sampling_space_description, 
                                                      total_population_size=total_population_size)


In [ ]:
difficult_people_population_factory.initialize_sampling_plan()
difficult_people_population_factory.sampling_plan

In [ ]:
difficult_people_population = difficult_people_population_factory.generate_people(population_size, 
                                                                                  agent_particularities="Very negative in all aspects, completely lacking any kind of enthusiasm or positivity.",
                                                                             post_processing_func=post_process_agent,
                                                                             verbose=True)

In [ ]:
profiler.profile(difficult_people_population)

In [ ]:
save_population(difficult_people_population, "./population/difficult_people_2/")

## Populations: Political compass sample

In [ ]:
political_commentators_sampling_space_description = describe_population_sampling_space("American", "./information/populations/usa.json")

def political_inclination(name):
    return f"Regarding political and economic beliefs, **must** be of **strong** {name} inclinations (beliefs, behaviors, preferences, personality, etc.)."

In [ ]:
authoritarian_leftwing_factory = TinyPersonFactory(sampling_space_description=political_commentators_sampling_space_description, total_population_size=total_population_size)
authoritarian_rightwing_factory = TinyPersonFactory(sampling_space_description=political_commentators_sampling_space_description, total_population_size=total_population_size)
libertarian_leftwing_factory = TinyPersonFactory(sampling_space_description=political_commentators_sampling_space_description, total_population_size=total_population_size)
libertarian_rightwing_factory = TinyPersonFactory(sampling_space_description=political_commentators_sampling_space_description, total_population_size=total_population_size)

In [ ]:
def aux_authoritarian_leftwing(agent):
    agent.import_fragment("./fragments/leftwing.agent.fragment.json")
    agent.import_fragment("./fragments/authoritarian.agent.fragment.json")

authoritarian_leftwing_sample = authoritarian_leftwing_factory.generate_people(small_population_size, 
                                               #agent_particularities=political_inclination("authoritarian leftwing"), 
                                               post_processing_func=aux_authoritarian_leftwing,
                                               verbose=True)

In [ ]:
profiler.profile(authoritarian_leftwing_sample)
save_population(authoritarian_leftwing_sample, "./population/political_compass_2/")

In [ ]:
def aux_authoritarian_rightwing(agent):
    agent.import_fragment("./fragments/rightwing.agent.fragment.json")
    agent.import_fragment("./fragments/authoritarian.agent.fragment.json")

authoritarian_rightwing_sample = authoritarian_rightwing_factory.generate_people(small_population_size, 
                                               #agent_particularities=political_inclination("authoritarian rightwing"), 
                                               post_processing_func=aux_authoritarian_rightwing,
                                               verbose=True)

In [ ]:
profiler.profile(authoritarian_rightwing_sample)
save_population(authoritarian_rightwing_sample, "./population/political_compass_2/")

In [ ]:
def aux_libertarian_leftwing(agent):
    agent.import_fragment("./fragments/leftwing.agent.fragment.json")
    agent.import_fragment("./fragments/libertarian.agent.fragment.json")

libertarian_leftwing_sample = libertarian_leftwing_factory.generate_people(small_population_size, 
                                               #agent_particularities=political_inclination("libertarian leftwing"),
                                               post_processing_func=aux_libertarian_leftwing,
                                               verbose=True)

In [ ]:
profiler.profile(libertarian_leftwing_sample)
save_population(libertarian_leftwing_sample, "./population/political_compass_2/")

In [ ]:
def aux_libertarian_rightwing(agent):
    agent.import_fragment("./fragments/rightwing.agent.fragment.json")
    agent.import_fragment("./fragments/libertarian.agent.fragment.json")

libertarian_rightwing_sample = libertarian_rightwing_factory.generate_people(small_population_size,
                                               #agent_particularities=political_inclination("libertarian rightwing"), 
                                               post_processing_func=aux_libertarian_rightwing,
                                               verbose=True)

In [ ]:
profiler.profile(libertarian_rightwing_sample)
save_population(libertarian_rightwing_sample, "./population/political_compass_2/")

## A company

In [ ]:
company_sampling_space_description = \
f""""
....
""" 

In [ ]:
company_population_factory = TinyPersonFactory(sampling_space_description=company_sampling_space_description, 
                                                      total_population_size=total_population_size)


In [ ]:
company_population_factory.initialize_sampling_plan()
company_population_factory.sampling_plan

In [ ]:
company_population = company_population_factory.generate_people(population_size, 
                                                                                  agent_particularities="",
                                                                             post_processing_func=post_process_agent,
                                                                             verbose=True)

In [ ]:
profiler.profile(company_population)

In [ ]:
save_population(difficult_people_population, "./population/company_2/")